In [2]:
import pandas as pd
import numpy as np

In [3]:
path = '/Users/tomcio/Documents/GitHub/CBS_SunCulture/data/'

In [14]:
df_payments = pd.read_csv(path + 'Payments.csv')
df_payments['expectedDate'] = pd.to_datetime(df_payments['expectedDate'] )
df_payments['timestampMade'] = pd.to_datetime(df_payments['timestampMade'] )
df_payments['expectedMonth'] = df_payments['expectedDate'].dt.month
df_payments['expectedYear'] = df_payments['expectedDate'].dt.year
# move expectedMonth and expectedYear right after expectedDate
df_payments.insert(df_payments.columns.get_loc('expectedDate'), 'expectedMonth', df_payments.pop('expectedMonth'))
df_payments.insert(df_payments.columns.get_loc('expectedDate'), 'expectedYear', df_payments.pop('expectedYear'))
df_payments

/var/folders/73/nqn3vmmj4nxdmfh1wmm5jvdr0000gn/T/ipykernel_5269/30843774.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_payments['expectedDate'] = pd.to_datetime(df_payments['expectedDate'] )
/var/folders/73/nqn3vmmj4nxdmfh1wmm5jvdr0000gn/T/ipykernel_5269/30843774.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_payments['timestampMade'] = pd.to_datetime(df_payments['timestampMade'] )


,paymentSequence,expectedAmount,expectedMonth,expectedYear,expectedDate,amount paid,timestampMade,Device ID
0,1,4599,8,2022,2022-08-25,4600,2022-08-24 17:26:00,869640055831054
1,1,4599,3,2022,2022-03-05,4600,2022-03-07 12:48:00,868328054319309
2,1,4599,2,2022,2022-02-05,4600,2022-02-07 13:54:00,869640055849940
3,1,4599,7,2022,2022-07-15,4600,2022-07-05 09:09:00,869640055781481
4,1,4599,4,2022,2022-04-15,4600,2022-04-11 07:40:00,869640055891835
...,...,...,...,...,...,...,...,...
764994,17,2649,9,2024,2024-09-05,100,2024-08-02 19:56:00,869640056972667
764995,17,2649,9,2024,2024-09-05,100,2024-08-04 19:23:00,869640056972667
764996,18,2649,10,2024,2024-10-05,100,2024-08-17 21:18:00,869640056972667
764997,18,2649,10,2024,2024-10-05,100,2024-08-20 21:23:00,869640056972667


In [13]:
expected_group = df_payments.groupby(['Device ID', 'expectedYear', 'expectedMonth'])['expectedAmount'].max().reset_index()
expected_group

,Device ID,expectedYear,expectedMonth,expectedAmount
0,000086I67714040,2024,8,2999
1,007DJ5051960481,2024,2,4299
2,007DJ5051960481,2024,3,4299
3,007DJ5051960481,2024,4,4299
4,007DJ5051960481,2024,5,4299
...,...,...,...,...
345625,BJG665052353207,2024,7,4299
345626,BJG665052353207,2024,8,4299
345627,XJD556065293053,2024,5,4299
345628,XJD556065293053,2024,6,4299


In [ ]:
df_payments['Month_Paid'] = df_payments['timestampMade'].dt.month
df_payments['Year_Paid'] = df_payments['timestampMade'].dt.year
paid_group = df_payments.groupby(['Device ID', 'Year_Paid', 'Month_Paid'])['amount paid'].sum().reset_index()
paid_group.rename(columns={'Month_Paid': 'Month'}, inplace=True)
paid_group.rename(columns={'Year_Paid': 'Year'}, inplace=True)
paid_group

,Device ID,Year,Month,amount paid
0,272909,2024,3,6000
1,272909,2024,7,17194
2,860666,2024,1,9800
3,860666,2024,2,9800
4,860666,2024,3,10000
...,...,...,...,...
296017,XJD556065293053,2024,4,230
296018,XJD556065293053,2024,5,7900
296019,XJD556065293053,2024,6,2000
296020,XJD556065293053,2024,7,6600


In [ ]:
df_combined = pd.merge(expected_group, paid_group, on=['Device ID', 'Year', 'Month'], how='left')
df_combined

,Device ID,Year,Month,expectedAmount,amount paid
0,272909,2024,6,4299,NaN
1,272909,2024,7,4299,17194.0
2,860666,2024,1,4599,9800.0
3,860666,2024,2,4599,9800.0
4,860666,2024,3,4599,10000.0
...,...,...,...,...,...
345625,BJG665052353207,2024,7,4299,3200.0
345626,BJG665052353207,2024,8,4299,1500.0
345627,XJD556065293053,2024,5,4299,7900.0
345628,XJD556065293053,2024,6,4299,2000.0


In [ ]:
# Crear la tabla para consolidar la información por Device ID
device_payment_info = []

# Agrupar por Device ID
grouped_df = df_combined.groupby('Device ID')

# Iterar por cada Device ID
for device_id, group in grouped_df:
    expected_amount_cumulative = group['expectedAmount'].sum()
    amount_paid_cumulative = group['amount paid'].sum()

    # Default? comparar si el monto esperado es mayor que el pagado
    is_default = 'Yes' if expected_amount_cumulative > amount_paid_cumulative else 'No'

    # Consolidar la información en la nueva tabla
    device_payment_info.append({
        'Device ID': device_id,
        'Expected Payments Cumulative': expected_amount_cumulative,
        'Amount Paid Cumulative': amount_paid_cumulative,
        'Default?': is_default
    })

# Crear el DataFrame final con la información consolidada
df_device_payment_info = pd.DataFrame(device_payment_info)
df_device_payment_info

,Device ID,Expected Payments Cumulative,Amount Paid Cumulative,Default?
0,272909,8598,17194.0,No
1,860666,36792,69600.0,No
2,35641109,20094,43549.0,No
3,665055244,32193,50590.0,No
4,869556065,8598,8600.0,No
...,...,...,...,...
20465,BJDII6065198484,4299,0.0,Yes
20466,BJDJ04064504714,16494,34699.0,No
20467,BJDJ04064698979,9898,9900.0,No
20468,BJG665052353207,34392,41100.0,No


In [ ]:
# Contar el total de dispositivos
total_devices = len(df_device_payment_info)

# Contar cuántos dispositivos están en default
devices_in_default = len(df_device_payment_info[df_device_payment_info['Default?'] == 'Yes'])

# Calcular el porcentaje
percentage_in_default = (devices_in_default / total_devices) * 100

# Mostrar el resultado
print(f"El {percentage_in_default:.2f}% de los dispositivos está en default.")

El 17.62% de los dispositivos está en default.


In [ ]:
df_iot = pd.read_csv('CSDirectTelemetry_Aggregated_New.csv')

In [ ]:
df_iot

,DeviceId,Year,Month,num_days_with_data,num_active_days,num_inactive_days,total_active_records,total_inactive_records,avg_RSSI,avg_PanelVoltage,avg_MotorCurrent,avg_MotorVoltage,avg_MotorSpeed,avg_MotorPower
0,868328054195702,2024,1,3,2,1,106,191,30.983165,35.570020,3.940104,11.661636,937.535354,136.967100
1,869604064508855,2024,1,31,22,9,3195,16755,19.797995,41.559197,1.315669,40.339038,304.541053,NaN
2,869640056869756,2024,1,31,0,31,0,19775,18.036612,39.723700,0.000000,0.000000,0.000000,0.000000
3,869640056937025,2024,1,29,4,25,817,15986,20.379158,39.516907,0.239725,1.198476,96.305660,8.678694
4,869640056981189,2024,1,19,1,18,77,12123,14.594426,40.113545,0.021611,0.132057,10.615082,0.752475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87808,867556041891675,2023,12,1,1,0,22,2,26.958333,37.245750,4.391250,19.372625,3312.000000,160.597269
87809,868328054304403,2023,12,1,1,0,1,48,18.571429,41.860755,0.607898,36.127816,18378.612245,24.521106
87810,868328054243825,2023,12,1,1,0,25,25,6.200000,35.554760,1.146620,11.361880,912.480000,40.880008
87811,868328054278862,2023,12,1,1,0,3,109,13.857143,40.909964,0.068750,0.464018,37.285714,2.661157


In [ ]:
device_signal_info = []

# Agrupar por Device ID
grouped_signal_df = df_iot.groupby('DeviceId')

# Iterar por cada Device ID
for device_id, group in grouped_signal_df:
    # Sumar total de num_distinct_days_with_signal
    total_days_with_signal = group['num_active_days'].sum()

    # Sumar total de num_days_no_signal
    total_days_no_signal = group['num_inactive_days'].sum()

    # Calcular el porcentaje de días con señal
    total_days = total_days_with_signal + total_days_no_signal


    # Solo incluir si el total de días es mayor o igual a 60
    if total_days >= 60:
        # Calcular el porcentaje de días con señal
        percentage_with_signal = (total_days_with_signal / total_days) * 100 if total_days > 0 else 0

        # Consolidar la información en la nueva tabla
        device_signal_info.append({
            'Device ID': device_id,
            'Total Days with Signal': total_days_with_signal,
            'Total Days without Signal': total_days_no_signal,
            'Percentage of Days with Signal': percentage_with_signal
        })

df_device_signal_info = pd.DataFrame(device_signal_info)
df_device_signal_info

,Device ID,Total Days with Signal,Total Days without Signal,Percentage of Days with Signal
0,860665052438933,215,8,96.412556
1,860665059716869,180,16,91.836735
2,860665059719095,39,197,16.525424
3,860665059722024,224,24,90.322581
4,860665059724111,88,7,92.631579
...,...,...,...,...
7464,869640056995916,93,1,98.936170
7465,869640056995924,153,74,67.400881
7466,869640056995965,118,61,65.921788
7467,869640056995973,4,134,2.898551


In [ ]:
df_device_payment_info['Device ID'] = df_device_payment_info['Device ID'].astype(str)
df_device_signal_info['Device ID'] = df_device_signal_info['Device ID'].astype(str)
df_device_default_analysis = pd.merge(df_device_payment_info,
                                      df_device_signal_info,
                                      on='Device ID',
                                      how='inner')

In [ ]:
df_device_default_analysis

,Device ID,Expected Payments Cumulative,Amount Paid Cumulative,Default?,Total Days with Signal,Total Days without Signal,Percentage of Days with Signal
0,860665052438933,47980,87471.0,No,215,8,96.412556
1,860665059716869,110376,149193.0,No,180,16,91.836735
2,860665059719095,45033,98250.0,No,39,197,16.525424
3,860665059724111,61586,162520.0,No,88,7,92.631579
4,860665059729698,73933,123900.0,No,148,84,63.793103
...,...,...,...,...,...,...,...
4019,869640056995866,34437,53540.0,No,180,54,76.923077
4020,869640056995916,26994,46000.0,No,93,1,98.936170
4021,869640056995924,105576,256152.0,No,153,74,67.400881
4022,869640056995965,48389,48389.0,No,118,61,65.921788


In [ ]:
df_device_default_analysis['Signal Percentile'] = pd.qcut(df_device_default_analysis['Percentage of Days with Signal'],
                                                         q=10,
                                                         labels=[f'{i*10}-{(i+1)*10}%' for i in range(10)])

# Función para calcular el porcentaje de dispositivos en default en cada percentil
def calculate_default_percentage(group):
    total_devices = len(group)
    devices_in_default = len(group[group['Default?'] == 'Yes'])
    return (devices_in_default / total_devices) * 100 if total_devices > 0 else 0

# Agrupar por los percentiles y calcular el porcentaje de default y el promedio de Percentage of Days with Signal
percentile_default_analysis = df_device_default_analysis.groupby('Signal Percentile').agg(
    Default_Percentage=('Default?', lambda x: (x == 'Yes').mean() * 100),
    Average_Percentage_Signal_Days=('Percentage of Days with Signal', 'mean')
).reset_index()

# Renombrar las columnas (opcional)
percentile_default_analysis.columns = ['Signal Percentile', 'Default Percentage', 'Average Percentage of Days with Signal']
df_percentile_default_analysis = pd.DataFrame(percentile_default_analysis)
df_percentile_default_analysis

,Signal Percentile,Default Percentage,Average Percentage of Days with Signal
0,0-10%,18.858561,7.316900
1,10-20%,14.179104,25.325323
2,20-30%,13.151365,40.638945
3,30-40%,11.940299,54.066474
4,40-50%,9.203980,66.155727
5,50-60%,12.935323,77.022098
6,60-70%,11.414392,85.438197
7,70-80%,12.935323,92.677894
8,80-90%,10.918114,97.062828
9,90-100%,13.432836,99.496102


In [ ]:
device_avg_active_records = []

# Agrupar los datos por Device ID
grouped_df = df_iot.groupby('DeviceId')

# Iterar por cada Device ID en el grupo
for device_id, group in grouped_df:
    # Contar cuántos meses distintos hay para ese Device ID
    num_months = group[['Year', 'Month']].drop_duplicates().shape[0]

    # Sumar todos los total_active_records para ese Device ID
    total_active_records_sum = group['total_active_records'].sum()

    # Calcular el promedio de total_active_records por mes
    avg_active_records_per_month = total_active_records_sum / num_months

    # Guardar el resultado en la lista
    device_avg_active_records.append({
        'Device ID': device_id,
        'Avg Active Records per Month': avg_active_records_per_month
    })

# Crear un nuevo DataFrame con los resultados
df_avg_active_records = pd.DataFrame(device_avg_active_records)

# Mostrar la tabla resultante
df_avg_active_records

,Device ID,Avg Active Records per Month
0,000000000000000,64.833333
1,860665052149084,26.250000
2,860665052438933,3836.400000
3,860665059716869,2539.100000
4,860665059717172,142.400000
...,...,...
13816,869640056995965,3507.700000
13817,869640056995973,37.000000
13818,869640056995981,140.000000
13819,869640056996013,1339.800000


In [ ]:
df_avg_active_records['Device ID'] = df_avg_active_records['Device ID'].astype(str)
df_device_active_records_default_analysis = pd.merge(df_device_payment_info,
                                      df_avg_active_records,
                                      on='Device ID',
                                      how='inner')
df_device_active_records_default_analysis

,Device ID,Expected Payments Cumulative,Amount Paid Cumulative,Default?,Avg Active Records per Month
0,860665052438933,47980,87471.0,No,3836.4
1,860665059716869,110376,149193.0,No,2539.1
2,860665059719095,45033,98250.0,No,93.7
3,860665059724111,61586,162520.0,No,694.7
4,860665059724186,8698,4349.0,Yes,0.0
...,...,...,...,...,...
5727,869640056995866,34437,53540.0,No,2637.8
5728,869640056995916,26994,46000.0,No,2668.5
5729,869640056995924,105576,256152.0,No,4393.9
5730,869640056995965,48389,48389.0,No,3507.7


In [ ]:
df_device_active_records_default_analysis['Signal Percentile'] = pd.qcut(df_device_active_records_default_analysis['Avg Active Records per Month'],
                                                         q=10,
                                                         labels=[f'{i*10}-{(i+1)*10}%' for i in range(10)])

# Función para calcular el porcentaje de dispositivos en default en cada percentil
def calculate_default_percentage(group):
    total_devices = len(group)
    devices_in_default = len(group[group['Default?'] == 'Yes'])
    return (devices_in_default / total_devices) * 100 if total_devices > 0 else 0

# Agrupar por los percentiles y calcular el porcentaje de default y el promedio de Percentage of Days with Signal
percentile_default_analysis = df_device_active_records_default_analysis.groupby('Signal Percentile').agg(
    Default_Percentage=('Default?', lambda x: (x == 'Yes').mean() * 100),
    Average_active_records_month=('Avg Active Records per Month', 'mean')
).reset_index()

# Renombrar las columnas (opcional)
percentile_default_analysis.columns = ['Signal Percentile', 'Default Percentage', 'Avg Active Records per Month']
df_percentile_default_analysis = pd.DataFrame(percentile_default_analysis)
df_percentile_default_analysis

,Signal Percentile,Default Percentage,Avg Active Records per Month
0,0-10%,29.790941,6.328019
1,10-20%,18.989547,65.564537
2,20-30%,15.909091,158.929974
3,30-40%,15.357766,275.334218
4,40-50%,16.230366,428.684085
5,50-60%,18.324607,665.828590
6,60-70%,14.956522,1037.233640
7,70-80%,15.061296,1647.629141
8,80-90%,15.706806,2725.740110
9,90-100%,15.853659,5612.678884
